In [1]:
import ee
import geemap
import pandas as pd
from datetime import datetime, timedelta
import csv


In [6]:
## Open buildings Data collection

In [3]:


# Initialize the Earth Engine library
ee.Initialize()

# Define the AOI using the given coordinates


aoi = ee.Geometry.Polygon(
    [[
        [31.53211417878892, 29.999910849582736],
        [31.548765332353373, 29.999910849582736],
        [31.548765332353373, 30.01180319662939],
        [31.53211417878892, 30.01180319662939],
        [31.53211417878892, 29.999910849582736]
    ]],
    None,  # No CRS needed for default WGS84
    False  # Non-geodesic
)

# Define the Image Collection and scale
image_collection = ee.ImageCollection('GOOGLE/Research/open-buildings-temporal/v1')
SCALE_M = 1  # Adjust for larger AOIs if needed

# Initialize a map
Map = geemap.Map()

# Prepare to export data
output_data = []

# Loop through the years and process data
for year in range(2016, 2024):
    epoch_s = (
        ee.Date(f'{year}-06-30', 'America/Los_Angeles')
        .millis()
        .divide(1000)
    )
    mosaic = image_collection.filter(ee.Filter.eq('inference_time_epoch_s', epoch_s)).mosaic()
    count = (
        mosaic.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=aoi,
            scale=SCALE_M,
            crs=aoi.projection()
        )
        .getNumber('building_fractional_count')
        .multiply(ee.Number(SCALE_M * 2).pow(2))
    )
    
    # Add the data to the export list
    output_data.append({"Year": year, "Building Count": count.getInfo()})

    # Add the layer to the map
    Map.addLayer(
        mosaic.select('building_presence'),
        {'min': 0, 'max': 1},
        str(year)
    )
    print(f'Building count for year {year}: {count.getInfo()}')

# Write the data to a local CSV file
output_file = "building_counts.csv"
with open(output_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["Year", "Building Count"])
    writer.writeheader()
    writer.writerows(output_data)

print(f"Data exported to {output_file}")

# Display the map
Map.centerObject(aoi, zoom=15)
Map



Building count for year 2016: 1825.5769204800604
Building count for year 2017: 2055.502466130043
Building count for year 2018: 2302.0262609896427
Building count for year 2019: 2605.7163150517827
Building count for year 2020: 2955.865561758267
Building count for year 2021: 3004.582146718046
Building count for year 2022: 3238.183144793663
Building count for year 2023: 3431.3405774719217
Data exported to building_counts.csv


Map(center=[30.005856994379606, 31.540439755577122], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Initialize the Earth Engine library
ee.Initialize()

# Define the starting AOI polygon (bottom-left corner of the region)
start_lon = 74.67792937305236
start_lat = 31.913394575025592
aoi_width = 0.01  # Longitude size of each AOI
aoi_height = 0.01  # Latitude size of each AOI

# Define the number of AOIs to generate in each direction
num_aoi_lon = 40  # Number of steps in the longitude direction
num_aoi_lat = 30  # Number of steps in the latitude direction

# Define the Image Collection and scale
image_collection = ee.ImageCollection('GOOGLE/Research/open-buildings-temporal/v1')
SCALE_M = 10

# Prepare data for export
results = []

# Generate AOIs and collect data
for i in range(num_aoi_lon):
    for j in range(num_aoi_lat):
        # Calculate the coordinates for the AOI
        current_lon = start_lon + i * aoi_width
        current_lat = start_lat + j * aoi_height
        
        aoi = ee.Geometry.Polygon(
            [[
                [current_lon, current_lat],
                [current_lon + aoi_width, current_lat],
                [current_lon + aoi_width, current_lat + aoi_height],
                [current_lon, current_lat + aoi_height],
                [current_lon, current_lat]
            ]]
        )
        
        # Get AOI coordinates for tracking
        aoi_coordinates = aoi.getInfo()['coordinates']
        
        # Create a dictionary for the AOI with the coordinates
        year_data = {"AOI Coordinates": str(aoi_coordinates)}
        
        # Loop through the years
        for year in range(2016, 2024):
            # Define the epoch time for the year
            epoch_s = (
                ee.Date(f'{year}-06-30', 'America/Los_Angeles')
                .millis()
                .divide(1000)
            )
            
            # Filter the image collection for the year and mosaic
            mosaic = image_collection.filter(ee.Filter.eq('inference_time_epoch_s', epoch_s)).mosaic()
            
            # Calculate the building count within the AOI
            count = (
                mosaic.reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=aoi,
                    scale=SCALE_M,
                    maxPixels=1e13
                )
                .getNumber('building_fractional_count')
            )
            
            # Convert count to the correct value
            try:
                count_value = count.getInfo()
                if count_value is None:  # Handle missing data
                    count_value = 0
            except:
                count_value = 0  # Handle errors
            
            # Scale the count to match the expected output
            count_value_scaled = count_value * 400  # Scaling factor based on expected values
            
            # Add the year and scaled building count to the dictionary
            year_data[year] = count_value_scaled
            
            print(f'Building count for AOI {aoi_coordinates}, year {year}: {count_value_scaled}')
        
        # Append the dictionary to the results
        results.append(year_data)

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
output_file = "large_aoi_building_counts.csv"
df.to_csv(output_file, index=False)

print(f"Data exported to {output_file}")


Building count for AOI [[[74.67792937305236, 31.913394575025592], [74.68792937305237, 31.913394575025592], [74.68792937305237, 31.923394575025593], [74.67792937305236, 31.923394575025593], [74.67792937305236, 31.913394575025592]]], year 2016: 27.14910233787176
Building count for AOI [[[74.67792937305236, 31.913394575025592], [74.68792937305237, 31.913394575025592], [74.68792937305237, 31.923394575025593], [74.67792937305236, 31.923394575025593], [74.67792937305236, 31.913394575025592]]], year 2017: 37.96509686366174
Building count for AOI [[[74.67792937305236, 31.913394575025592], [74.68792937305237, 31.913394575025592], [74.68792937305237, 31.923394575025593], [74.67792937305236, 31.923394575025593], [74.67792937305236, 31.913394575025592]]], year 2018: 51.85979516116134
Building count for AOI [[[74.67792937305236, 31.913394575025592], [74.68792937305237, 31.913394575025592], [74.68792937305237, 31.923394575025593], [74.67792937305236, 31.923394575025593], [74.67792937305236, 31.91339

In [7]:
# Initialize Earth Engine
ee.Initialize()

# Define the date range
start_date = '2016-01-01'
end_date = '2023-12-31'

# Load ERA5 datasets
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('mean_2m_air_temperature') \
    .filter(ee.Filter.date(start_date, end_date))

era5_tp = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('total_precipitation') \
    .filter(ee.Filter.date(start_date, end_date))

era5_2d = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('dewpoint_2m_temperature') \
    .filter(ee.Filter.date(start_date, end_date))

era5_mslp = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('mean_sea_level_pressure') \
    .filter(ee.Filter.date(start_date, end_date))

era5_sp = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('surface_pressure') \
    .filter(ee.Filter.date(start_date, end_date))

era5_u_wind_10m = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .select('u_component_of_wind_10m') \
    .filter(ee.Filter.date(start_date, end_date))

# Convert surface pressure from Pa to hPa
era5_sp = era5_sp.map(lambda image: image.divide(100).set(
    'system:time_start', image.get('system:time_start')))

# Visualization parameters
vis_tp = {
    'min': 0.0,
    'max': 0.1,
    'palette': ['ffffff', '00ffff', '0080ff', 'da00ff', 'ffa400', 'ff0000']
}

vis_2mt = {
    'min': 250,
    'max': 320,
    'palette': [
        '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff', '00ff80',
        '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00', 'ffb000', 'ffa400',
        'ff4f00', 'ff2500', 'ff0a00', 'ff00ff'
    ]
}

vis_wind = {
    'min': 0,
    'max': 30,
    'palette': [
        'ffffff', 'ffff71', 'deff00', '9eff00', '77b038', '007e55', '005f51',
        '004b51', '013a7b', '023aad'
    ]
}

vis_pressure = {
    'min': 500,
    'max': 1150,
    'palette': [
        '01ffff', '058bff', '0600ff', 'df00ff', 'ff00ff', 'ff8c00', 'ff8c00'
    ]
}

# Create a map
map = geemap.Map(center=(22.2, 21.2), zoom=2)

# Add layers to the map
map.addLayer(era5_tp.mean(), vis_tp, 'Daily total precipitation (2016-2023)')
map.addLayer(era5_2d.mean(), vis_2mt, 'Daily mean 2m dewpoint temperature (2016-2023)')
map.addLayer(era5_2mt.mean(), vis_2mt, 'Daily mean 2m air temperature (2016-2023)')
map.addLayer(era5_u_wind_10m.mean(), vis_wind, 'Daily mean 10m u-component of wind (2016-2023)')
map.addLayer(era5_sp.mean(), vis_pressure, 'Daily mean surface pressure (2016-2023)')

# Display the map
map


Map(center=[22.2, 21.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…